In [ ]:
!pip -q install gradio

In [ ]:
import gradio as gr
import subprocess

test_case_bytes = b"def add(x, y):\n  return x + y\n\nresult = add(2, 3)\nprint(result)"
test_case_string = test_case_bytes.decode('utf-8')  # Decode assuming UTF-8 encoding

def evaluate_code(code):

  """
  Evaluates the submitted Python code against a test case.

  Args:
      code (str): The Python code submitted by the student.

  Returns:
      str: A message indicating if the code passed or failed the test case,
          along with the expected and actual outputs.
  """
  test_case = """
def add(x, y):
  return x + y

# Test the add function
result = add(2, 3)
print(result)
"""
# Define test case as bytes

  try:
    # Combine student and test case code
    ## combined_code = code + test_case_string
    combined_code = code

    # Execute the code in a safe environment with timeout
    result = subprocess.run(
        ["python"],
        input=combined_code,
        capture_output=True,
        text=True,
        timeout=15  # Adjust timeout as needed
    )

    # Check if the output matches the expected output
    expected_output = "5"
    if result.returncode == 0 and result.stdout.strip() == expected_output:
      return "Code passed! Expected output: " + expected_output + ". Your output: " + result.stdout.strip()
    else:
      return "Code failed the test case. Expected output: " + expected_output + ". Your output: " + result.stdout.strip() if result.stdout else "No output"

  except subprocess.TimeoutExpired:
    return "Timeout Error: Code execution timed out."
  except Exception as e:
    return f"Error: {e}"

interface = gr.Interface(
  fn=evaluate_code,
  inputs="textarea",
  outputs="text",
  title="Python Code Tester",
  description="Enter your Python code below and see if it passes the test case."
)

interface.launch(debug=True)
